# 모두의 딥러닝
## 20장 전이 학습을 통해 딥러닝의 성능 극대화하기

- 이미지 같은경우 충분한 학습데이터를 모으는것이 쉽지않음
- 이럴때 쓸수있는 방법을 두가지를 알아볼것

- 각 data폴더안에 train, test폴더에
- normal 정상뇌사진 ad 치매뇌사진이
- train 80장씩 test 60장씩 있음

In [1]:
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
import matplotlib.pyplot as plt
import os

os.environ['KMP_DUPLICATE_LIB_OK']="TRUE"

In [2]:
BATCH_SIZE = 5

In [3]:
# 특징을 미묘하게 바꿔서
# 이미지 데이터의 수를 확장할 때 효과적으로 사용할 수 있다
train_datagen = ImageDataGenerator(rescale=1./255,          # 주어진 이미지의 크기를 설정합니다.
                                  horizontal_flip=True,     # 수평 대칭 이미지를 50% 확률로 만들어 추가합니다.
                                  width_shift_range=0.1,    # 전체 크기의 15% 범위에서 좌우로 이동합니다.
                                  height_shift_range=0.1,   # 마찬가지로 위, 아래로 이동합니다.
                                  #rotation_range=5,        # 정해진 각도만큼 회전시킵니다.
                                  #shear_range=0.7,         # 좌표 하나를 고정시키고 나머지를 이동시킵니다.
                                  #zoom_range=1.2,          # 확대 또는 축소시킵니다.
                                  #vertical_flip=True,      # 수직 대칭 이미지를 만듭니다.
                                  #fill_mode='nearest'      # 빈 공간을 채우는 방법입니다. nearest 옵션은 가장 비슷한 색으로 채우게 됩니다.
                                  )

train_generator = train_datagen.flow_from_directory(
       './data/train',   # 학습셋이 있는 폴더의 위치
       target_size=(150, 150), # 이미지크기
       batch_size=BATCH_SIZE,
       class_mode='binary') # 치매/정상 이진 분류이므로 바이너리 모드로 실행

Found 160 images belonging to 2 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)  

test_generator = test_datagen.flow_from_directory(
       './data/test',   # 테스트셋이 있는 폴더의 위치
       target_size=(150, 150),
       batch_size=BATCH_SIZE,
       class_mode='binary')

Found 120 images belonging to 2 classes.


In [5]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(32, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 activation (Activation)     (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 72, 72, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                    

In [6]:
model.compile(loss='binary_crossentropy', 
              # 디폴트 learing_rate는 0.001 보폭을 더 줄인거임
              optimizer=optimizers.Adam(learning_rate=0.0002), 
              metrics=['accuracy'])

In [7]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
history = model.fit(
    train_generator, epochs=100, 
    validation_data=test_generator, 
    callbacks=[early_stopping]
)

Epoch 1/100
32/32 [==============================] - 3s 69ms/step - loss: 0.7158 - accuracy: 0.4563 - val_loss: 0.6877 - val_accuracy: 0.5000
Epoch 2/100
32/32 [==============================] - 2s 62ms/step - loss: 0.6930 - accuracy: 0.5063 - val_loss: 0.6832 - val_accuracy: 0.6833
Epoch 3/100
32/32 [==============================] - 2s 60ms/step - loss: 0.6805 - accuracy: 0.6125 - val_loss: 0.6845 - val_accuracy: 0.5000
Epoch 4/100
32/32 [==============================] - 2s 59ms/step - loss: 0.6710 - accuracy: 0.6062 - val_loss: 0.6509 - val_accuracy: 0.5583
Epoch 5/100
32/32 [==============================] - 2s 60ms/step - loss: 0.6498 - accuracy: 0.6750 - val_loss: 0.6600 - val_accuracy: 0.5167
Epoch 6/100
32/32 [==============================] - 2s 60ms/step - loss: 0.6101 - accuracy: 0.7250 - val_loss: 0.7624 - val_accuracy: 0.5000
Epoch 7/100
32/32 [==============================] - 2s 59ms/step - loss: 0.6196 - accuracy: 0.6625 - val_loss: 0.6268 - val_accuracy: 0.5917
Epoch 

In [ ]:
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

# 그래프로 표현
x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c="red", label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()